In [48]:
import tensorflow as tf
# import tensorflow_io as tfio
# from tensorflow import keras
from tensorflow.keras import Sequential,layers
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
# import keras
from keras import backend as K
from src import data
from src import error_function
from src import util
from src.data import DataSet

from sklearn.model_selection import train_test_split

import IPython.display as ipd
import os 
import glob 
import pdb

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
print(tf.__version__)

2.5.0


In [72]:
dataset_object = DataSet("../data/merged_outputtrain/*/*.wav", val_split=0.2)

In [73]:
train_dataset, val_dataset = dataset_object.get_data()

In [74]:
batch_data = []
batch_label = []
counter = 0
for audio,label in train_dataset.as_numpy_iterator():
    counter += 1
    if(counter == 10):
        batch_data = audio
        batch_label = label

In [76]:
batch_label

array([1, 4, 6, 9, 1, 5, 3, 4, 8, 0, 8, 8, 2, 4, 8, 5, 1, 0, 2, 5, 6, 3,
       9, 9, 6, 8, 3, 9, 3, 0, 9, 6], dtype=int16)

In [3]:
def obtain_label(filename):
    return filename.split("/")[-2]

In [4]:
# # @tf.function
# def parse_function(filename, label):
# #     audio_sample, sample_rate = tf.audio.decode_wav(filename)    
#     audio_binary = tf.io.read_file(filename)
# #     audio_decoded = tf.contrib.ffmpeg.decode_audio(
# #         audio_binary,
# #         file_format="wav",
# #         samples_per_second=16000,
# #         channel_count=1
# #     )
# #     audio_path = tf.keras.utils.get_file('test.wav', filename)
# #     audio = tfio.audio.AudioIODataset(audio_path)
#     audio,sr = tf.audio.decode_wav(audio_binary)
# #     print(audio.shape)
#     pdb.set_trace()
#     return tf.cast(audio, tf.float32),label
# #https://github.com/faroit/python_audio_loading_benchmark

In [5]:
def parse_function(filename):
    filepath = bytes.decode(filename.numpy())
    audio, sample_rate = sf.read(filepath)
    label = obtain_label(filepath)
    return tf.cast(audio, tf.float32),label

In [6]:
def parse_function_wrapper(filename):
    return tf.py_function(func=parse_function, inp=[filename], Tout=(tf.float32, tf.string))

In [7]:
def data_generator():
    filenames = glob.glob('../data/merged_outputtrain/*/*')
    labels = list(map(obtain_label, filenames))
    for filename,label in zip(filenames,labels):
        yield filename,label

In [8]:
def select_random_excerpt(file,label): #length can be changed..
    excerpt_duration=1
    excerpt_duration_frames = excerpt_duration * 16000
    limit = 80000 - excerpt_duration_frames
    start = np.random.randint(0,limit)
    return tf.slice(tf.squeeze(file), [start], [excerpt_duration_frames-1]),label

In [9]:
#stft(data,n_fft=400,hop_length=160,window=signal.windows.hann):
    #return np.abs(librosa.stft(data, n_fft=n_fft,hop_length=hop_length,window=window)).T
#frame_length = n_fft
#frame_step = hop_length
def stft_tf(audio_tensor, label):
    data = audio_tensor.numpy()
    stft = util.stft(data)
    label = int(bytes.decode(label.numpy()))
    return tf.cast(util.stft(data), tf.float32),tf.convert_to_tensor(label, dtype=tf.int16)

In [10]:
def stft_wrapper(audiofile, label): 
    audio, label = tf.py_function(func=stft_tf, inp=[audiofile,label], Tout=(tf.float32,tf.int16) )
    return audio, label


In [16]:
def reshape(audiofile, label):
    return tf.reshape(audiofile,shape=(1,100,201)), label

In [29]:
filenames = tf.io.gfile.glob("../data/merged_outputtrain/*/*.wav")#glob.glob('../data/merged_outputtrain/*/*')
labels = list( map(obtain_label, filenames) )
X_train, X_val, y_train, y_val = train_test_split(filenames, labels,
                                                  train_size=0.8, 
                                                  random_state=42)

In [18]:
print(X_train[:10])
print(y_train[:10])

['../data/merged_outputtrain/7/7_1186.wav', '../data/merged_outputtrain/2/2_341.wav', '../data/merged_outputtrain/5/5_1024.wav', '../data/merged_outputtrain/9/9_848.wav', '../data/merged_outputtrain/7/7_786.wav', '../data/merged_outputtrain/4/4_553.wav', '../data/merged_outputtrain/5/5_64.wav', '../data/merged_outputtrain/4/4_213.wav', '../data/merged_outputtrain/2/2_91.wav', '../data/merged_outputtrain/2/2_1291.wav']
['7', '2', '5', '9', '7', '4', '5', '4', '2', '2']


In [30]:
print(X_train[:10])
print(y_train[:10])

['../data/merged_outputtrain/3/3_682.wav', '../data/merged_outputtrain/1/1_40.wav', '../data/merged_outputtrain/4/4_1913.wav', '../data/merged_outputtrain/6/6_1505.wav', '../data/merged_outputtrain/3/3_312.wav', '../data/merged_outputtrain/9/9_1648.wav', '../data/merged_outputtrain/4/4_1253.wav', '../data/merged_outputtrain/9/9_438.wav', '../data/merged_outputtrain/1/1_370.wav', '../data/merged_outputtrain/1/1_1300.wav']
['3', '1', '4', '6', '3', '9', '4', '9', '1', '1']


In [19]:
# filenames = glob.glob('../data/merged_outputtrain/*/*')
# labels = list( map(obtain_label, filenames) )
# dataset = tf.data.Dataset.from_tensor_slices( (filenames,labels) )

dataset= tf.data.Dataset.list_files(X_train)
# dataset = tfio.audio.AudioIODataset( (filenames[0],labels[0]) ) causes kernel crash!!!
# dataset = tfio.audio.AudioIOTensor( filenames[0], dtype=tf.float64)
dataset = dataset.map(parse_function_wrapper, num_parallel_calls=4)
dataset = dataset.map(select_random_excerpt, num_parallel_calls=4)
dataset = dataset.map(stft_wrapper, num_parallel_calls=4)
dataset = dataset.map(reshape, num_parallel_calls=4)
dataset = dataset.batch(32)
dataset = dataset.prefetch(1)

In [20]:
batch_data = []
batch_label = []
counter = 0
for audio,label in dataset.as_numpy_iterator():
    counter += 1
    if(counter == 10):
        batch_data = audio
        batch_label = label

In [46]:
initializer = tf.keras.initializers.VarianceScaling(
    scale=1.0, mode="fan_avg", distribution="uniform", seed=None
)

In [47]:
model = Sequential(
    [
        layers.ZeroPadding2D(padding=((0,0),(0,0)), batch_input_shape=(None, 1,500,201),name="zero1",data_format="channels_last"),
        layers.Conv2D(64,kernel_size=(3,3),strides=(1,1),name="conv1",activation="relu",kernel_initializer=initializer,data_format="channels_first"),
        layers.Conv2D(32,kernel_size=(3,3),strides=(1,1),name="conv2",activation="relu",kernel_initializer=initializer,data_format="channels_first"),
        layers.MaxPool2D(pool_size=(3,3),strides=(3,3),trainable=True,name="pool1",data_format="channels_first"),
        layers.Conv2D(128,kernel_size=(3,3),strides=(1,1),name="conv3",activation="relu",kernel_initializer=initializer,data_format="channels_first"),
        layers.Conv2D(64,kernel_size=(3,3),strides=(1,1),name="conv4",activation="relu",kernel_initializer=initializer, data_format="channels_first"),
        layers.MaxPool2D(pool_size=(3,3),strides=(3,3),trainable=True,name="pool2",data_format="channels_first"),
        layers.Dropout(0.5, trainable=True, name="dropout_1"),
        layers.Permute((2,1,3),trainable=True,name="permute_1"),
        layers.Reshape((53,-1),trainable=True,name="reshape_1"),
        layers.LSTM(40, return_sequences=True,kernel_initializer=initializer,name="lstm_1",batch_input_shape=(None, None, 1280)),
        layers.MaxPool1D(pool_size=(2,),strides=(2,),trainable=True, name="maxpooling1d_1", data_format="channels_last"),
        layers.Flatten(name="flatten1",trainable=True),
        layers.Dense(11,activation="linear",kernel_initializer=initializer,trainable=True,name="dense_1", batch_input_shape=(None, 1040)),
        layers.Softmax(trainable=True)
    ]
)

In [48]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [49]:
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.sparse_categorical_crossentropy
             )

In [50]:
model.fit(dataset)

InvalidArgumentError:  Default MaxPoolingOp only supports NHWC on device type CPU
	 [[node sequential/pool1/MaxPool (defined at <ipython-input-50-d28793719248>:1) ]] [Op:__inference_train_function_74811]

Function call stack:
train_function
